# [자연어처리]
# 4주차(4-2). BOW 기반의 문서 분류(3)


# 202002961 김현주

# 이제 Colab 사용에 모든 수강생분들이 실행에는 어느정도 익숙해 졌을 거라 생각됩니다!
* (벌써 4주차 입니다. 시간 빠르죠?)
* **키보드**(+마우스)를 조금 더 사용해보도록 하겠습니다.
> ## 중간 중간 **'### ... 이 부분을 완성하시오'** 라는 부분의 코드를 완성해 보기 바랍니다. (강의 자료를 참고하여 '???' 부분을 작성해 보시기 바랍니다.)





LMS에서 daum_movie_review.csv 파일을 다운로드하여, 실습을 수행하기 바랍니다!

# 한글 문서의 분류
### 입력 값으로 리뷰가 들어오면, 어떤 영화에 대한 리뷰인지 예측하는 알고리즘을 만들어 보자!
* '신과함께, 택시운전사, 인피니티 워, 범죄도시, 곤지암, 라라랜드, 코코' 중 하나로 분류
* **입력**: 몰입할수밖에 없다. 어렵게 생각할 필요없다. 내가 전투에 참여한듯 손에 땀이남.
* **출력**: 인피니티 워

In [ ]:
import pandas as pd
df = pd.read_csv('./daum_movie_review.csv')
df.head(5)

,review,rating,date,title
0,돈 들인건 티가 나지만 보는 내내 하품만,1,2018.10.29,인피니티 워
1,몰입할수밖에 없다. 어렵게 생각할 필요없다. 내가 전투에 참여한듯 손에 땀이남.,10,2018.10.26,인피니티 워
2,이전 작품에 비해 더 화려하고 스케일도 커졌지만.... 전국 맛집의 음식들을 한데 ...,8,2018.10.24,인피니티 워
3,이 정도면 볼만하다고 할 수 있음!,8,2018.10.22,인피니티 워
4,재미있다,10,2018.10.20,인피니티 워


In [ ]:
df.tail(5)

,review,rating,date,title
14720,어른들을 위한 동화 정말 오랜만에 좋은 애니를 보았습니다 가족의 소중...,10,2018.01.12,코코
14721,디즈니는 못해도 본전은 한다.,7,2018.01.12,코코
14722,가족을 위한 영화... 괜찮은 영화.~~~,8,2018.01.12,코코
14723,간만에 제대로 잘짜여진 각본의 영화를 봤네 여운이 아직도 남아~어른을 위한 애니~,10,2018.01.12,코코
14724,한국개봉을 눈빠지게 기다린 보람이있다 깨우치는게 많은 영화,10,2018.01.12,코코


In [ ]:
df.title.value_counts()

,count
title,
신과함께,4947
택시운전사,2322
인피니티 워,2042
범죄도시,1939
곤지암,1547
라라랜드,1150
코코,778


In [ ]:
df.shape

(14725, 4)

In [ ]:
from sklearn.model_selection import train_test_split

# split data and labels into a training and a test set
# 비율(test_size)을 지정하지 않으면 75:25로 분할됨
X_train, X_test, y_train, y_test = train_test_split(df.review, df.title, random_state=0)

print('#Train set size:', len(X_train)) # 실제로 몇개의 특성이 사용되었는지 확인
print('#Test set size:', len(X_test))

#Train set size: 11043
#Test set size: 3682


In [ ]:
! pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 26.9 MB/s eta 0:00:00


In [ ]:
from konlpy.tag import Okt # konlpy에서 Twitter 형태소 분석기를 import

okt = Okt()

print(okt.morphs(X_train[1])) # 둘째 리뷰에 대해 형태소 단위로 tokenize
print(okt.nouns(X_train[1])) # 둘째 리뷰에서 명사만 추출

['몰입', '할수밖에', '없다', '.', '어렵게', '생각', '할', '필요없다', '.', '내', '가', '전투', '에', '참여', '한', '듯', '손', '에', '땀', '이남', '.']
['몰입', '생각', '내', '전투', '참여', '듯', '손', '땀', '이남']


In [ ]:
# 약간의 시간이 소요됩니다! (약 1분 30초)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Okt 형태소분석기에서 명사만 추출하는 함수를 tokenizer로 이용
tfidf = TfidfVectorizer(tokenizer=okt.nouns, max_features=2000, min_df=5, max_df=0.5)

X_train_tfidf = tfidf.fit_transform(X_train) # train data 변환 -> tfidf vector
X_test_tfidf = tfidf.transform(X_test) # test data 변환 -> tfidf vector

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
# logistic regression 분류기 선언
LR_clf = LogisticRegression()
# 분류기 학습
LR_clf.fit(X_train_tfidf, y_train)

print('#Train set score: {:.3f}'.format(LR_clf.score(X_train_tfidf, y_train))) # train data 예측정확도
print('#Test set score: {:.3f}'.format(LR_clf.score(X_test_tfidf, y_test))) # test data 예측정확도

#Train set score: 0.756
#Test set score: 0.694


In [ ]:
import numpy as np

def top10_features(classifier, vectorizer, categories):
    feature_names = np.asarray(vectorizer.get_feature_names_out())
    for i, category in enumerate(categories):
        # 역순으로 정렬하기 위해 계수에 음수를 취해서 정렬 후 앞에서부터 10개의 값을 반환
        top10 = np.argsort(-classifier.coef_[i])[:10]
        # 카테고리와 영향이 큰 특성 10개를 출력
        print("%s: %s" % (category, ", ".join(feature_names[top10])))

In [ ]:
top10_features(LR_clf, tfidf, sorted(df.title.unique()))

곤지암: 공포영화, 공포, 귀신, 곤지암, 무서움, 소리, 기담, 개, 공포물, 장난
라라랜드: 음악, 뮤지컬, 꿈, 사랑, 노래, 엔딩, 여운, 인생, 재즈, 환상
범죄도시: 마동석, 윤계상, 조선족, 액션, 조연, 조폭, 범죄, 마블리, 형사, 폭력
신과함께: 원작, 차태현, 웹툰, 신파, 김동욱, 판타지, 신, 천만, 지옥, 하정우
인피니티 워: 마블, 노스, 어벤져스, 히어로, 번역, 토르, 편, 결말, 다음, 영웅
코코: 디즈니, 애니, 겨울왕국, 감동, 코코, 픽사, 애니메이션, 가족, 어른, 노래
택시운전사: 광주, 송강호, 역사, 전두환, 택시, 가슴, 진실, 민주주의, 감동, 추격


In [ ]:
# 첫 번째 리뷰에 대한 예측 값 살펴보기
print('첫 번째 리뷰 텍스트: ', X_test.iloc[0])
print('특성으로 추출된 토큰: ', okt.nouns(X_test.iloc[0]))
predict = LR_clf.predict(X_test_tfidf[0])
print('예측된 영화 제목: ', predict[0])
print('실제 영화 제목: ', y_test.iloc[0])

첫 번째 리뷰 텍스트:  오랜만에 잼나는 영화 봤습니다.  다음에 더 재미있는 영화 기대하겠습니다.
특성으로 추출된 토큰:  ['만', '영화', '다음', '더', '영화']
예측된 영화 제목:  신과함께
실제 영화 제목:  범죄도시


In [ ]:
for i in range(10):
    print(i, '번째 리뷰 텍스트: ', X_test.iloc[i])
    print('특성으로 추출된 토큰: ', okt.nouns(X_test.iloc[i]))
    predict = LR_clf.predict(X_test_tfidf[i])
    print('예측된 영화 제목: ', predict[0])
    print('실제 영화 제목: ', y_test.iloc[i])
    print()

0 번째 리뷰 텍스트:  오랜만에 잼나는 영화 봤습니다.  다음에 더 재미있는 영화 기대하겠습니다.
특성으로 추출된 토큰:  ['만', '영화', '다음', '더', '영화']
예측된 영화 제목:  신과함께
실제 영화 제목:  범죄도시

1 번째 리뷰 텍스트:  조연들이 눈에 박힌다. 간만에 집중 ㅎ
특성으로 추출된 토큰:  ['조연', '눈', '간만', '집중']
예측된 영화 제목:  범죄도시
실제 영화 제목:  범죄도시

2 번째 리뷰 텍스트:  대감동을 선사. 인사이드 아웃을 잇는 픽사의 감동스토리. 신과함께의 멕시코판이라고나할까요??
특성으로 추출된 토큰:  ['감동', '선사', '인사', '이드', '아웃', '픽사', '감동', '스토리', '신', '의', '멕시코', '판이', '나', '요']
예측된 영화 제목:  코코
실제 영화 제목:  코코

3 번째 리뷰 텍스트:  돈이 안아까웠던 영화ᆞᆞ  정말 좋았다
특성으로 추출된 토큰:  ['돈', '영화', '정말']
예측된 영화 제목:  신과함께
실제 영화 제목:  신과함께

4 번째 리뷰 텍스트:  역시 김용화감독이 영화는 잘 만들어요. 이제 VFX 제작 부문도 헐리우드 수준 이상입니다.
특성으로 추출된 토큰:  ['역시', '김용화', '감독', '영화', '이제', '제작', '부문', '헐리우드', '수준', '이상']
예측된 영화 제목:  신과함께
실제 영화 제목:  신과함께

5 번째 리뷰 텍스트:  민주화를 위해 힘써주신 분들께 감사하는 마음으로 살아야겠다.
특성으로 추출된 토큰:  ['민주화', '위해', '분', '마음']
예측된 영화 제목:  택시운전사
실제 영화 제목:  택시운전사

6 번째 리뷰 텍스트:  잠만 자다 왔음
특성으로 추출된 토큰:  ['잠']
예측된 영화 제목:  신과함께
실제 영화 제목:  신과함께

7 번째 리뷰 텍스트:  오랜만에 잼있고 좋은 영화를 봤다
특성으로 추출된 토큰:  ['만', '영화']
예측된 영화 제목:  신과함께
실제 영화 제목:  

---

# **[[실습 4-2-1]]** 다른 분류기(RidgeClassifier)로 분류해보기
1. RidgeClassifier (alpha=0.1)를 이용하여 분류할 경우, Train set score와 Test set score는 얼마인가?
2. 이 데이터 셋에서는 valdation에 대한 최적의 alpha는 몇인가? 0.1과 10 사이에서 0.1씩 증가시키며 찾아보시오.
  * Hint: 4-1의 내용을 참고
  ```
  from sklearn.model_selection import train_test_split
  # 8:2의 비율로 나누기
  X_train_ridge, X_val_ridge, y_train_ridge, y_val_ridge = train_test_split(X_train_tfidf, y_train, test_size=0.2, random_state=42)
  ```
  ```
  max_score = 0
  max_alpha = 0
  for alpha in np.arange(0.1, 10, 0.1): # alpha를 0.1부터 10까지 0.1씩 증가
      ridge_clf = RidgeClassifier(alpha=alpha) # 릿지 분류기 선언
      ridge_clf.fit(X_train_ridge, y_train_ridge) # 학습
      score = ridge_clf.score(X_val_ridge, y_val_ridge) # 검정 데이터셋에 대해 정확도를 측정
      if score > max_score: # 정확도가 이전의 정확도 최대값보다 크면 최대값을 변경한다.
          max_score = score
          max_alpha = alpha
  print('Max alpha {:.3f} at max validation score {:.3f}'.format(max_alpha, max_score))
  ```
3. 최적의 alpha 값을 이용하여 분류할 경우, Train set score와 Test set score는 얼마인가?


In [ ]:
# 코드를 작성하시오

In [ ]:
from sklearn.linear_model import RidgeClassifier

ridge_clf = RidgeClassifier(alpha=0.1) # 릿지 분류기 선언
ridge_clf.fit(X_train_tfidf, y_train) # train data로 학습

print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

Train set score: 0.788
Test set score: 0.679


In [ ]:
from sklearn.model_selection import train_test_split
# 8:2의 비율로 나누기
X_train_ridge, X_val_ridge, y_train_ridge, y_val_ridge = train_test_split(X_train_tfidf, y_train, test_size=0.2, random_state=42)

In [ ]:
max_score = 0
max_alpha = 0

for alpha in np.arange(0.1, 10, 0.1): # alpha를 0.1부터 10까지 0.1씩 증가
 ridge_clf = RidgeClassifier(alpha=alpha) # 릿지 분류기 선언
 ridge_clf.fit(X_train_ridge, y_train_ridge) # 학습
 score = ridge_clf.score(X_val_ridge, y_val_ridge) # 검정 데이터셋에 대해 정확도를 측정
 if score > max_score: # 정확도가 이전의 정확도 최대값보다 크면 최대값을 변경한다.
     max_score = score
     max_alpha = alpha
print('Max alpha {:.3f} at max validation score {:.3f}'.format(max_alpha, max_score))

Max alpha 1.100 at max validation score 0.696


In [ ]:
from sklearn.linear_model import RidgeClassifier

ridge_clf = RidgeClassifier(alpha=1.1) # 릿지 분류기 선언
ridge_clf.fit(X_train_tfidf, y_train) # train data로 학습

print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

Train set score: 0.778
Test set score: 0.693


(내용 작성)
* (0.1과 10사이에서) 최적의 alpha 값은 무엇인가?
* 해당 alpha 값을 이용하여 학습했을 때, Train set score와 Test set score는?

최적의 alpha 값은 1.1이며, 이 경우의 Train set score은 0.778,
Test set score은 0.693이 나왔다.

---

# [파일] -> [다운로드] -> [.ipynb 다운로드]